## Intro to Jupyter notebooks 📖
Jupyter-notebooks are online documents that combine programming langauges with text, so you can easily share and explain your code with others.

Each notebook contains boxes or cells that have either code or text. 

Running cell refers to activating the code/text within a cell. To run a cell, you enter the cell and either select the play symbol (▶️) at the top of the notebook or press shift + enter at the same time. 

Have a go by running this text cell and the next cell,  which will explain how complete the 360Giving data request with this notebook!

## Initial set-up required to use 360Giving notebook 🌍🎁

Some initial is required before using this notebook, **YOU MUST edit and run** the following two sections first...
1. Import packages (using the 2nd cell below that has 'import pandas as pd' ) - 
- **if this doesn't work**, then you should open 'python_libraries' notebook, run the cell there (this will install the necessary tools to run the notebook)
- then rerun the import cell in this notebook.
2. Change the effective year variables to match the effective year that the data is requested for. The financial year variables are found in the 4th cell below.


After completing these three steps, you can produce the 360Giving data report! 

Select the 'Run' button on Jupyter tab and then select **'Run All Cells'** - Tada! You should have the data request appear as a csv file in the output folder.

### 1. Importing packages and necssary function ⚙️

In [1]:
import os 
import pyodbc
import pandas as pd

def yield_rows(cursor):
    """
    Args:
        the cursor

    Yields:
        list(dict): List of dictionary with rows of the resulting query
        with format {"column": "row"}
    """
    column_names = [column for column, *_ in cursor.description]
    for row in cursor:
        yield {column: value for column, value in zip(column_names, row)}
        

### 2. Effective year variables 📅
In cell/block below, we enter the effective year start/end dates and store them as variables. 
The variables essential store the dates in the notebooks memory so it later be used as a filter for grants.
**IMPORTANT**: Please edit the effective_year start and end date for the date range you are looking for, otherwise you will get the wrong output!!!

In [2]:
#Effective date filter 

#FYI - FY22 = Oct 2021 to Sept 2022 = 2021/22, FY21 = Oct 2020 to Sep 2021 = 2020/21. 

#Effective  - please adjust
effective_year_start = "2005-10-01"
effective_year_end = "2022-12-31"

effective_year_start = pd.to_datetime(effective_year_start)
effective_year_end = pd.to_datetime(effective_year_end)

### Connecting and Joinning Fortytwo tables 

The following section connects three databases from Fortytwo/the data warehouse and joins them as one dataframe. Each table contains information we need for the 360Giving data request.

#### Connecting to GrantFamily table - Fortytwo database 🔗

In [3]:
#Connection variables  to server and database -  must be defined first 
server = "WT-AWS-42SQL"
database = 'Fortytwo'
table ="dbo.GrantFamily"



#General connection code
connection = f'DRIVER={{ODBC Driver 17 for SQL Server}};' \
                 f'SERVER={server};' \
                 f'DATABASE={database};' \
                 f'Trusted_connection=yes;'

In [4]:
#Fields of choice from Grants Family - as list so python can understand it 
Grant_Family_fields_of_choice = ['Family Financial Year', 'Family Parent Reference', 'Family Administrative Lead Applicant', 'Family Co-Applicant(s)', 'Family Other Lead Applicant(s)', 'Family Co-Applicant Organisation(s)', 'Family Other Lead Applicant Administering Organisation(s)', 'Family Master Grant Type Name', 'Family Administrative Lead Applicant Administering Organisation', 'Family Title', 'Family Synopsis', 'Family Start Date', 'Family Effective Date', 'Family End Date', 'Family Current Award Wellcome Value £', 'Family Current Award £']

#Did not get Applicant Surname, Region or Country, Public Summary Title, Public Summmary 

Grant_Family_fields_of_choice_sql_ver = "["+'],['.join(Grant_Family_fields_of_choice)+"]"
Grant_Family_query = f'''SELECT {Grant_Family_fields_of_choice_sql_ver} FROM {table}'''

In [5]:
Grant_Family_connector_ms = pyodbc.connect(connection)
# Cursor allows you to query the database you're connected 
Grant_Family_cursor = Grant_Family_connector_ms.execute(Grant_Family_query)


In [6]:
GrantFamily_original = pd.DataFrame(yield_rows(Grant_Family_cursor))

In [7]:
GrantFamily_original

,Family Financial Year,Family Parent Reference,Family Administrative Lead Applicant,Family Co-Applicant(s),Family Other Lead Applicant(s),Family Co-Applicant Organisation(s),Family Other Lead Applicant Administering Organisation(s),Family Master Grant Type Name,Family Administrative Lead Applicant Administering Organisation,Family Title,Family Synopsis,Family Start Date,Family Effective Date,Family End Date,Family Current Award Wellcome Value £,Family Current Award £
0,2004/05,076762/Z/05/Z,Prof Anthony Lee,Dr Malcolm East,None,University of Southampton,None,Project Grant,University of Southampton,How a mechanosensitive channel couples to the ...,Bacteria use mechanosensitive channels to help...,2005-08-01,2005-04-30,2008-07-31,149753.95,149753.95
1,2004/05,076764/Z/05/Z,Prof Gavin Vinson,"Anonymised, Anonymised",None,Queen Mary University of London,None,Project funding: Inactive scheme,Queen Mary University of London,Nucleo-cytoplasmic shuttling of nuclear recept...,Soluble proteins are not confined statically t...,2005-03-01,2005-05-04,2008-02-29,NaN,NaN
2,2004/05,076765/Z/05/Z,Dr Matthew Hodgkin,None,None,None,None,Project Grant,University of Warwick,Caspase-mediated cleavage of human phospholipa...,No Data Entered,2006-02-01,2005-05-04,2009-01-31,NaN,NaN
3,2004/05,076772/Z/05/Z,Dr Donald Ward,Prof Daniela Riccardi,None,Cardiff University,None,Project Grant,University of Manchester,Regulation of parathyroid hormone secretion an...,This project will examine how the Calcium Sens...,2005-05-01,2005-04-26,2008-04-30,NaN,NaN
4,2015/16,203968/Z/16/Z,Mr Istvan Kleijn,None,None,None,None,PhD Studentship (Basic),Imperial College London,Imperial College London - Theoretical Systems ...,Imperial College London - Theoretical Systems ...,2016-10-01,2016-09-30,2021-07-01,141554.46,141554.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112733,2022/23,227478/Z/23/Z,Dr David Oduori,None,None,None,None,Career Development Award,Maasai Mara University,Innovative Leishmaniasis Control Model using O...,The project aims to establish a One Health sur...,2023-05-31,2023-05-16,2031-05-30,NaN,NaN
112734,2022/23,227479/Z/23/Z,Dr Vyacheslav Karolis,None,None,None,None,Career Development Award,King's College London,Tracing the foundations of high-level cognitio...,It is increasingly appreciated that aberration...,2023-09-01,2023-05-16,2031-08-31,NaN,NaN
112735,2022/23,227480/Z/23/Z,Dr Jennifer Bizley,None,None,None,None,Career Development Award,University College London,How does the brain map sounds into the world?,"In hearing, spatial information must be comput...",2023-09-01,2023-05-16,2031-08-31,NaN,NaN
112736,2022/23,227481/Z/23/Z,Prof David Lalloo,Prof Ymkje Sienstra,None,Liverpool School of Tropical Medicine,None,Discretionary Award,Liverpool School of Tropical Medicine,Snakebite Research Alliance- Initial Activities,The 2019 WHO roadmap aims to halve snakebite d...,2023-01-06,2023-09-30,2023-07-05,66110.00,66110.00


#### Connecting to Apps+Grants table - Fortytwo database 🔗

In [8]:
#Connection to server and database 
server2 = "WT-AWS-42SQL"
database2 = 'Fortytwo'
table2 = "dbo.ApplicationsAndGrantDetails"

#General connection code
connection2 = f'DRIVER={{ODBC Driver 17 for SQL Server}};' \
                 f'SERVER={server2};' \
                 f'DATABASE={database2};' \
                 f'Trusted_connection=yes;'

#Fields of choice from Grants Family - as list so python can understand it 
Apps_Grants_fields_of_choice = ["Reference", "Status", "Outcome", "Termination Reason", "Administering Organisation UK Region", "Administering Organisation Country", "Public Summary Title", "Public Summary", "Department", "Team", 'Area', 'Partnership Name']

Apps_Grants_fields_of_choice_sql_ver = "["+'],['.join(Apps_Grants_fields_of_choice)+"]"
Apps_Grants_query = f'''SELECT {Apps_Grants_fields_of_choice_sql_ver} FROM {table2}'''


Apps_Grants_connector_ms = pyodbc.connect(connection2)
Apps_Grants_cursor = Apps_Grants_connector_ms.execute(Apps_Grants_query)

In [9]:
Apps_Grants_original = pd.DataFrame(yield_rows(Apps_Grants_cursor))

In [10]:
Apps_Grants_original

,Reference,Status,Outcome,Termination Reason,Administering Organisation UK Region,Administering Organisation Country,Public Summary Title,Public Summary,Department,Team,Area,Partnership Name
0,006593/Z/76/Z,Closed,Funded,None,Greater London,United Kingdom,None,None,None,None,None,None
1,006598/Z/76/Z,Closed,Funded,None,Greater London,United Kingdom,None,None,None,None,None,None
2,006599/Z/77/Z,Closed,Funded,None,Greater London,United Kingdom,None,None,None,None,None,None
3,006601/Z/76/Z,Closed,Funded,None,Wales,United Kingdom,None,None,None,None,None,None
4,006602/Z/76/Z,Closed,Funded,None,Greater London,United Kingdom,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
130251,227478/Z/23/Z,Closed,Withdrawn,None,International,Kenya,None,None,Research Programmes,None,Discovery Research,None
130252,227479/Z/23/Z,Under Review,None,None,Greater London,United Kingdom,None,None,Research Programmes,None,Discovery Research,None
130253,227480/Z/23/Z,Under Review,None,None,Greater London,United Kingdom,None,None,Research Programmes,None,Discovery Research,None
130254,227481/Z/23/Z,Active,Funded,None,North West,United Kingdom,None,None,Research Programmes,None,Translation & Portfolio Integration,None


#### Connecting to Apps+Grants table - Fortytwo_Denormalised database 🔗

In [11]:
#Connection to server and database 
server3 = "WT-AWS-42SQL"
database3 = 'Fortytwo_Denormalised'
table3 = "WTGT.ApplicationsAndGrantDetails"

#General connection code
connection3 = f'DRIVER={{ODBC Driver 17 for SQL Server}};' \
                 f'SERVER={server3};' \
                 f'DATABASE={database3};' \
                 f'Trusted_connection=yes;'

#Fields of choice from Grants Family - as list so python can understand it 
apps_grants_fields_of_choice3 = ["Reference", "Indirect Funding"]

apps_grants_fields_of_choice_sql_ver3 = "["+'],['.join(apps_grants_fields_of_choice3)+"]"
apps_grants_query3 = f'''SELECT {apps_grants_fields_of_choice_sql_ver3} FROM {table3}'''


apps_grants_connector_ms3 = pyodbc.connect(connection3)
apps_grants_cursor3 = apps_grants_connector_ms3.execute(apps_grants_query3)

In [12]:
Apps_Grants_Denormalised = pd.DataFrame(yield_rows(apps_grants_cursor3))

In [13]:
Apps_Grants_Denormalised

,Reference,Indirect Funding
0,061860/Z/00/C,None
1,074124/Z/04/L,None
2,087415/Z/08/Z,None
3,086017/Z/08/Z,None
4,086167/Z/08/Z,None
...,...,...
130251,084776/Z/08/Z,None
130252,085058/Z/08/Z,None
130253,085894/Z/08/Z,None
130254,085576/Z/08/Z,None


##### The two cells below are done to combine the data into two different columns as one new columns that matches 360Giving data standard 

e.g. the new column Other Applicant(s) is the same as combining 'Family Other Lead Applicant(s)' and 'Family Co-Applicant(s)'

In [14]:
#Concat ([Family Other Lead Applicant(s)],[Family Co-Applicant(s)]) AS [Other Applicant(s)]
GrantFamily_original['Other Applicant(s)'] = GrantFamily_original['Family Other Lead Applicant(s)'].astype(str) + ',' + GrantFamily_original['Family Co-Applicant(s)'].astype(str)
#Concat (['Family Other Lead Applicant Administering Organisation(s)'],['Family Co-Applicant Organisation(s)']) AS 'Other Applicant(s) Organisations]
GrantFamily_original['Other Applicant(s) Organisations'] = GrantFamily_original['Family Other Lead Applicant Administering Organisation(s)'].astype(str) + ',' + GrantFamily_original['Family Co-Applicant Organisation(s)'].astype(str)


In [15]:
#using lambda to replace every 'None, ' or 'None' to just empty space - basically replace all versions of None with ''
GrantFamily_original['Other Applicant(s)'] = GrantFamily_original['Other Applicant(s)'].apply(lambda x: x.replace('None,', '').replace('None', ''))
GrantFamily_original['Other Applicant(s) Organisations'] = GrantFamily_original['Other Applicant(s) Organisations'].apply(lambda x: x.replace('None,', '').replace('None', ''))


In [16]:
#Inner join turned out to better - left join gave me two odd grants 
GrantFamily_AppsGrant_Merged_df = GrantFamily_original.merge(Apps_Grants_original.merge(Apps_Grants_Denormalised, on='Reference', how='inner'), left_on = 'Family Parent Reference', right_on = 'Reference', how='inner')


In [17]:
GrantFamily_AppsGrant_Merged_df

,Family Financial Year,Family Parent Reference,Family Administrative Lead Applicant,Family Co-Applicant(s),Family Other Lead Applicant(s),Family Co-Applicant Organisation(s),Family Other Lead Applicant Administering Organisation(s),Family Master Grant Type Name,Family Administrative Lead Applicant Administering Organisation,Family Title,...,Termination Reason,Administering Organisation UK Region,Administering Organisation Country,Public Summary Title,Public Summary,Department,Team,Area,Partnership Name,Indirect Funding
0,2004/05,076762/Z/05/Z,Prof Anthony Lee,Dr Malcolm East,None,University of Southampton,None,Project Grant,University of Southampton,How a mechanosensitive channel couples to the ...,...,None,South East,United Kingdom,None,None,None,None,None,None,None
1,2004/05,076764/Z/05/Z,Prof Gavin Vinson,"Anonymised, Anonymised",None,Queen Mary University of London,None,Project funding: Inactive scheme,Queen Mary University of London,Nucleo-cytoplasmic shuttling of nuclear recept...,...,None,Greater London,United Kingdom,None,None,None,None,None,None,None
2,2004/05,076765/Z/05/Z,Dr Matthew Hodgkin,None,None,None,None,Project Grant,University of Warwick,Caspase-mediated cleavage of human phospholipa...,...,None,West Midlands,United Kingdom,None,None,None,None,None,None,None
3,2004/05,076772/Z/05/Z,Dr Donald Ward,Prof Daniela Riccardi,None,Cardiff University,None,Project Grant,University of Manchester,Regulation of parathyroid hormone secretion an...,...,None,North West,United Kingdom,None,None,None,None,None,None,None
4,2015/16,203968/Z/16/Z,Mr Istvan Kleijn,None,None,None,None,PhD Studentship (Basic),Imperial College London,Imperial College London - Theoretical Systems ...,...,None,Greater London,United Kingdom,None,None,Strategy,TNL - Science,Transition & Legacy,Not Applicable,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112731,2022/23,227478/Z/23/Z,Dr David Oduori,None,None,None,None,Career Development Award,Maasai Mara University,Innovative Leishmaniasis Control Model using O...,...,None,International,Kenya,None,None,Research Programmes,None,Discovery Research,None,None
112732,2022/23,227479/Z/23/Z,Dr Vyacheslav Karolis,None,None,None,None,Career Development Award,King's College London,Tracing the foundations of high-level cognitio...,...,None,Greater London,United Kingdom,None,None,Research Programmes,None,Discovery Research,None,None
112733,2022/23,227480/Z/23/Z,Dr Jennifer Bizley,None,None,None,None,Career Development Award,University College London,How does the brain map sounds into the world?,...,None,Greater London,United Kingdom,None,None,Research Programmes,None,Discovery Research,None,None
112734,2022/23,227481/Z/23/Z,Prof David Lalloo,Prof Ymkje Sienstra,None,Liverpool School of Tropical Medicine,None,Discretionary Award,Liverpool School of Tropical Medicine,Snakebite Research Alliance- Initial Activities,...,None,North West,United Kingdom,None,None,Research Programmes,None,Translation & Portfolio Integration,None,None


### Date Filters applied 
Here we apply the effective year start and end date as filters on the database 

In [18]:
GrantFamily_AppsGrant_Merged_df['Family Start Date'] = pd.to_datetime(GrantFamily_AppsGrant_Merged_df['Family Start Date'])
GrantFamily_AppsGrant_Merged_df['Family End Date'] = pd.to_datetime(GrantFamily_AppsGrant_Merged_df['Family End Date'])
GrantFamily_AppsGrant_Merged_df['Family Effective Date'] = pd.to_datetime(GrantFamily_AppsGrant_Merged_df['Family Effective Date'])

In [19]:
GrantFamily_AppsGrant_Merged_df['Family Start Date'] = GrantFamily_AppsGrant_Merged_df['Family Start Date'].dt.tz_localize(None)
GrantFamily_AppsGrant_Merged_df['Family End Date'] = GrantFamily_AppsGrant_Merged_df['Family End Date'].dt.tz_localize(None)
GrantFamily_AppsGrant_Merged_df['Family Effective Date'] = GrantFamily_AppsGrant_Merged_df['Family Effective Date'].dt.tz_localize(None)

In [20]:
GrantFamily_AppsGrant_Merged_Date = GrantFamily_AppsGrant_Merged_df[(GrantFamily_AppsGrant_Merged_df['Family Effective Date'] >= effective_year_start) & (GrantFamily_AppsGrant_Merged_df['Family Effective Date'] <= effective_year_end)]


### Filters to match Research Funding's exclusion criteria

Research Funding has specifc criteria for what grants can be shared publicly. The filters below remove any grants that don't match these criteria. Above each filter is a short explaination as to why it's required.


In [21]:
#Using a for loop to strip many str text: I also realised that many columns names had empty spaces within the string so Antonio showed me how to use a for loop to cleans it
columns_to_clean = ['Outcome', 'Status', 'Termination Reason', 'Status']

for column in columns_to_clean:
   GrantFamily_AppsGrant_Merged_Date[column] = GrantFamily_AppsGrant_Merged_Date[column].str.strip()

/var/folders/ym/nt50cg2d4z79v6hgwgns0cvr0000gq/T/ipykernel_11455/1642951446.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GrantFamily_AppsGrant_Merged_Date[column] = GrantFamily_AppsGrant_Merged_Date[column].str.strip()


Filter_1:
- This filter gives a list of all grants that were funded that have had their award letters sent out and can therefore be publicised externally

In [22]:
filter_1 = (GrantFamily_AppsGrant_Merged_Date['Outcome'] == 'Funded') & (GrantFamily_AppsGrant_Merged_Date['Status'].isin(['Pending Start Date', 'Active','Complete','Closed']))


filter_2:
- gives a list of all grants that were awarded but that finished early and that can be publicised externally

In [23]:
filter_2 = (GrantFamily_AppsGrant_Merged_Date['Outcome'] == 'Terminated') & (GrantFamily_AppsGrant_Merged_Date['Termination Reason'].isin(['By applicant - activity incomplete', 'By applicant - activity complete', 'By WT - record terminated', 'Transfer']))


So, when we apply filter_1 and 2 to the wellcome_internal database, we will select grants that match the criteria for Filter 1 Or Filter 2!

filter_3: excludes specific grant types for different reasons e.g.
- the CEA, IRS and RE are personal awards not for research
- the OA awards are funds given out after a grant has finished to allow open access publishing of further papers arising from a closed grant.
- Beit grants were taken over by us when Beit stopped running their own grant programme

In [24]:
filter_3 = (~GrantFamily_AppsGrant_Merged_Date['Family Master Grant Type Name'].isin(['Clinical Excellence Award','International Recruitment Supplement','Recruitment Enhancement','Open Access for Terminated Award','Open Access for Terminated Awards','Beit Memorial Fellowship']))


filter_4:
- excludes grants where an internal decision has been made not to publish the data, this is usually due to commercial sensitivity but occasionally for animal/primate research. These grants typically contain ‘do not publish’ in the ‘Public Summary Title’.

In [25]:
GrantFamily_AppsGrant_Merged_Date['Public Summary Title'] = GrantFamily_AppsGrant_Merged_Date['Public Summary Title'].fillna('')
filter_4 = ~GrantFamily_AppsGrant_Merged_Date['Public Summary Title'].str.lower().str.contains("do not publish")


/var/folders/ym/nt50cg2d4z79v6hgwgns0cvr0000gq/T/ipykernel_11455/1705555620.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GrantFamily_AppsGrant_Merged_Date['Public Summary Title'] = GrantFamily_AppsGrant_Merged_Date['Public Summary Title'].fillna('')


filter_5: Each year, the Research Funding team manually remove a list of grants for specific reasons.
- E.g. removal of duplicate grant records where there has been a transfer record
- PhD programme grants where the funding was given to the student rather than given out on the project.
- The reason why each grant was excluded is recorded within the ‘manual exclude’ excel file.

In [26]:
manual_exclude = pd.read_excel('manual_exclude/excludes_without_transfers.xlsx').sort_index()
#using strip attribute to clear all empty text

manual_exclude['Reference'] = manual_exclude['Reference'].str.strip()




In [27]:
filter_5_manual = (~GrantFamily_AppsGrant_Merged_Date['Reference'].isin(manual_exclude['Reference']))
GrantFamily_AppsGrant_Merged_Date[filter_5_manual]

,Family Financial Year,Family Parent Reference,Family Administrative Lead Applicant,Family Co-Applicant(s),Family Other Lead Applicant(s),Family Co-Applicant Organisation(s),Family Other Lead Applicant Administering Organisation(s),Family Master Grant Type Name,Family Administrative Lead Applicant Administering Organisation,Family Title,...,Termination Reason,Administering Organisation UK Region,Administering Organisation Country,Public Summary Title,Public Summary,Department,Team,Area,Partnership Name,Indirect Funding
4,2015/16,203968/Z/16/Z,Mr Istvan Kleijn,None,None,None,None,PhD Studentship (Basic),Imperial College London,Imperial College London - Theoretical Systems ...,...,None,Greater London,United Kingdom,,None,Strategy,TNL - Science,Transition & Legacy,Not Applicable,None
6,2015/16,205267/Z/16/Z,Prof Paul Workman,None,None,None,None,Open Access Award,Institute of Cancer Research,Open access block grant 2016/17,...,None,Greater London,United Kingdom,,None,None,None,None,The Charity Open Access Fund,None
7,2016/17,208626/Z/17/Z,Prof Trevor McMillan,None,None,None,None,Vacation Scholarships,Keele University,Vacation Scholarships 2017 - Keele University,...,None,West Midlands,United Kingdom,,None,None,None,None,Not applicable,None
13,2017/18,209755/Z/17/Z,Prof Leszek Borysiewicz,None,None,None,None,Open Access Award,University of Cambridge,Open Access Awards 2017/18,...,None,East of England,United Kingdom,,None,None,None,None,The Charity Open Access Fund,None
14,2017/18,213291/Z/18/Z,Prof Paul Layzell,None,None,None,None,Vacation Scholarships,"Royal Holloway, University of London","Vacation Scholarships 2018 - Royal Holloway, U...",...,By applicant - no payments made,South East,United Kingdom,,None,None,None,None,Not applicable,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112410,2022/23,227153/Z/23/Z,Dr Remco van de Pas,"Prof Hermann Lotze-Campen, Prof Ilona Otto",None,"University of Graz, Potsdam Institute for Clim...",None,Directed Call - full,Deutsche Allianz Klimawandel und Gesundheit e....,A Nutritional Transformation for Climate and H...,...,None,International,Germany,,None,Research Programmes,None,Climate & Health,None,None
112411,2022/23,227154/Z/23/Z,Prof Mary Brennan,"Dr Kirsteen Shields, Dr Simone Lamont-Black, P...",None,"University of Cambridge, University of Edinbur...",None,Directed Call - full,University of Edinburgh,"Building for health centred, net zero aligned ...",...,None,Scotland,United Kingdom,,None,Research Programmes,None,Climate & Health,None,None
112416,2022/23,227161/Z/23/Z,Dr Michael Greenstone,None,None,None,None,Directed Call - full,University of Chicago,Novel Data Technology Applications to Transfor...,...,None,International,United States,,None,Research Programmes,None,Climate & Health,None,None
112420,2022/23,227164/Z/23/Z,Prof Margherita Ferrante,None,None,None,None,Directed Call - full,University of Catania,One-HeaLth clImate obServatory: a global inter...,...,None,International,Italy,,None,Research Programmes,None,Climate & Health,None,None


In [28]:
GrantFamily_AppsGrant_Merged_RF_Filtered = GrantFamily_AppsGrant_Merged_Date[(filter_1 | filter_2) & filter_3 & filter_4 & filter_5_manual]



In [29]:
GrantFamily_AppsGrant_Merged_RF_Filtered

,Family Financial Year,Family Parent Reference,Family Administrative Lead Applicant,Family Co-Applicant(s),Family Other Lead Applicant(s),Family Co-Applicant Organisation(s),Family Other Lead Applicant Administering Organisation(s),Family Master Grant Type Name,Family Administrative Lead Applicant Administering Organisation,Family Title,...,Termination Reason,Administering Organisation UK Region,Administering Organisation Country,Public Summary Title,Public Summary,Department,Team,Area,Partnership Name,Indirect Funding
4,2015/16,203968/Z/16/Z,Mr Istvan Kleijn,None,None,None,None,PhD Studentship (Basic),Imperial College London,Imperial College London - Theoretical Systems ...,...,None,Greater London,United Kingdom,,None,Strategy,TNL - Science,Transition & Legacy,Not Applicable,None
6,2015/16,205267/Z/16/Z,Prof Paul Workman,None,None,None,None,Open Access Award,Institute of Cancer Research,Open access block grant 2016/17,...,None,Greater London,United Kingdom,,None,None,None,None,The Charity Open Access Fund,None
7,2016/17,208626/Z/17/Z,Prof Trevor McMillan,None,None,None,None,Vacation Scholarships,Keele University,Vacation Scholarships 2017 - Keele University,...,None,West Midlands,United Kingdom,,None,None,None,None,Not applicable,None
13,2017/18,209755/Z/17/Z,Prof Leszek Borysiewicz,None,None,None,None,Open Access Award,University of Cambridge,Open Access Awards 2017/18,...,None,East of England,United Kingdom,,None,None,None,None,The Charity Open Access Fund,None
16,2018/19,215036/Z/18/Z,Prof Alice Gast,None,None,None,None,Open Access Award,Imperial College London,Open Access (COAF) Award 2018/19,...,None,Greater London,United Kingdom,,None,None,None,None,The Charity Open Access Fund,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112007,2021/22,226727/Z/22/Z,Prof Neil Ferguson,"Dr Katy Gaythorpe, Dr Ilaria Dorigatti, Dr Car...",None,"Imperial College London, University of Cambridge",None,Discretionary Award,Imperial College London,Vaccine Impact Modelling Consortium (VIMC 2.0)...,...,None,Greater London,United Kingdom,,None,Research Programmes,None,Infectious Disease,None,None
112009,2021/22,226734/Z/22/Z,Mr David Glauser,None,None,None,None,Discretionary Award,Panorama Global,Civil Society Consultation and Engagement in t...,...,None,International,United States,,None,Strategy,None,Policy,None,None
112013,2021/22,226738/Z/22/Z,Ms Desta Lakew,None,None,None,None,Discretionary Award,Amref Health Africa,"Thought Leadership, Advocacy, and Partnership ...",...,None,International,Kenya,,None,Strategy,None,Policy,None,None
112044,2021/22,226767/Z/22/Z,Miss Ekaterina Roth,None,None,None,None,Discretionary Award,World Economic Forum,World Economic Forum Strategic Partnership Grant,...,None,International,Switzerland,,None,Strategy,"See ""Government Relations & Strategic Partners...",Government Relations & Strategic Partnerships,None,None


### 360Giving Standard: Changing  titles of columns 
- Some of our column names need to be changed to match the 360Giviing criteria 
- some additional columns need to be created from scratch such as Months or Public versions of the title/synopis. More details are found below

In [30]:
 
GrantFamily_AppsGrant_Merged_RF_Filtered['Internal ID'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Family Parent Reference']

GrantFamily_AppsGrant_Merged_RF_Filtered['Lead Applicant'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Family Administrative Lead Applicant']

GrantFamily_AppsGrant_Merged_RF_Filtered['Region'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Administering Organisation UK Region']

GrantFamily_AppsGrant_Merged_RF_Filtered['Recipient Org:Name'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Family Administrative Lead Applicant Administering Organisation']

GrantFamily_AppsGrant_Merged_RF_Filtered['Recipient Org:Country'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Administering Organisation Country']

GrantFamily_AppsGrant_Merged_RF_Filtered['Grant Programme:Title'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Family Master Grant Type Name']

GrantFamily_AppsGrant_Merged_RF_Filtered['Planned Dates:Start Date'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Family Start Date']

GrantFamily_AppsGrant_Merged_RF_Filtered['Planned Dates:End Date'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Family End Date']

GrantFamily_AppsGrant_Merged_RF_Filtered['Amount Awarded'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Family Current Award Wellcome Value £']

GrantFamily_AppsGrant_Merged_RF_Filtered['Award Date'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Family Effective Date']

GrantFamily_AppsGrant_Merged_RF_Filtered['Partnership Value'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Family Current Award £']

GrantFamily_AppsGrant_Merged_RF_Filtered['Financial Year'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Family Financial Year']



/var/folders/ym/nt50cg2d4z79v6hgwgns0cvr0000gq/T/ipykernel_11455/4176799599.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GrantFamily_AppsGrant_Merged_RF_Filtered['Internal ID'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Family Parent Reference']
/var/folders/ym/nt50cg2d4z79v6hgwgns0cvr0000gq/T/ipykernel_11455/4176799599.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GrantFamily_AppsGrant_Merged_RF_Filtered['Lead Applicant'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Family Administrative Lead 

In [31]:
GrantFamily_AppsGrant_Merged_RF_Filtered['Recipient Org:Identifier'] = '360G-Wellcome-ORG:' + GrantFamily_AppsGrant_Merged_RF_Filtered['Family Administrative Lead Applicant Administering Organisation'].astype(str)


/var/folders/ym/nt50cg2d4z79v6hgwgns0cvr0000gq/T/ipykernel_11455/4077312988.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GrantFamily_AppsGrant_Merged_RF_Filtered['Recipient Org:Identifier'] = '360G-Wellcome-ORG:' + GrantFamily_AppsGrant_Merged_RF_Filtered['Family Administrative Lead Applicant Administering Organisation'].astype(str)


In [32]:
GrantFamily_AppsGrant_Merged_RF_Filtered['Identifier'] =  '360G-Wellcome-' + GrantFamily_AppsGrant_Merged_RF_Filtered['Internal ID'].astype(str).apply(lambda x: x.replace('/','_'))


/var/folders/ym/nt50cg2d4z79v6hgwgns0cvr0000gq/T/ipykernel_11455/52302574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GrantFamily_AppsGrant_Merged_RF_Filtered['Identifier'] =  '360G-Wellcome-' + GrantFamily_AppsGrant_Merged_RF_Filtered['Internal ID'].astype(str).apply(lambda x: x.replace('/','_'))


In [33]:
GrantFamily_AppsGrant_Merged_RF_Filtered['Recipient Org:Identifier'] = '360G-Wellcome-ORG:' + (GrantFamily_AppsGrant_Merged_RF_Filtered['Recipient Org:Country'].astype(str).apply(lambda x: x.replace(' ','-')))



/var/folders/ym/nt50cg2d4z79v6hgwgns0cvr0000gq/T/ipykernel_11455/1119793051.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GrantFamily_AppsGrant_Merged_RF_Filtered['Recipient Org:Identifier'] = '360G-Wellcome-ORG:' + (GrantFamily_AppsGrant_Merged_RF_Filtered['Recipient Org:Country'].astype(str).apply(lambda x: x.replace(' ','-')))


In [34]:
GrantFamily_AppsGrant_Merged_RF_Filtered['Recipient Org:Identifier']

4         360G-Wellcome-ORG:United-Kingdom
6         360G-Wellcome-ORG:United-Kingdom
7         360G-Wellcome-ORG:United-Kingdom
13        360G-Wellcome-ORG:United-Kingdom
16        360G-Wellcome-ORG:United-Kingdom
                        ...               
112007    360G-Wellcome-ORG:United-Kingdom
112009     360G-Wellcome-ORG:United-States
112013             360G-Wellcome-ORG:Kenya
112044       360G-Wellcome-ORG:Switzerland
112452     360G-Wellcome-ORG:United-States
Name: Recipient Org:Identifier, Length: 16808, dtype: object

In [35]:
GrantFamily_AppsGrant_Merged_RF_Filtered['Funding Org:Name'] = 'The Wellcome Trust'


/var/folders/ym/nt50cg2d4z79v6hgwgns0cvr0000gq/T/ipykernel_11455/1389769615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GrantFamily_AppsGrant_Merged_RF_Filtered['Funding Org:Name'] = 'The Wellcome Trust'


In [36]:
GrantFamily_AppsGrant_Merged_RF_Filtered['Funding Org:Identifier'] = 'GB-CHC-210183'


/var/folders/ym/nt50cg2d4z79v6hgwgns0cvr0000gq/T/ipykernel_11455/2537978405.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GrantFamily_AppsGrant_Merged_RF_Filtered['Funding Org:Identifier'] = 'GB-CHC-210183'


In [37]:
GrantFamily_AppsGrant_Merged_RF_Filtered['Currency'] = 'GBP'

/var/folders/ym/nt50cg2d4z79v6hgwgns0cvr0000gq/T/ipykernel_11455/843554842.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GrantFamily_AppsGrant_Merged_RF_Filtered['Currency'] = 'GBP'


In [38]:
GrantFamily_AppsGrant_Merged_RF_Filtered['Funding Org:Department'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Area']

/var/folders/ym/nt50cg2d4z79v6hgwgns0cvr0000gq/T/ipykernel_11455/2418245958.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GrantFamily_AppsGrant_Merged_RF_Filtered['Funding Org:Department'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Area']


In [39]:
GrantFamily_AppsGrant_Merged_RF_Filtered['Funding Org:Team'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Team']

/var/folders/ym/nt50cg2d4z79v6hgwgns0cvr0000gq/T/ipykernel_11455/1876974657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GrantFamily_AppsGrant_Merged_RF_Filtered['Funding Org:Team'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Team']


In [40]:
GrantFamily_AppsGrant_Merged_RF_Filtered['For Regrant Type'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Indirect Funding']

/var/folders/ym/nt50cg2d4z79v6hgwgns0cvr0000gq/T/ipykernel_11455/2372963690.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GrantFamily_AppsGrant_Merged_RF_Filtered['For Regrant Type'] = GrantFamily_AppsGrant_Merged_RF_Filtered['Indirect Funding']


#### Public vs Avaliable title 

For Grants that have either a public title or summary, we cannot use title or synopsis but the public alternative 

I create two functions that will return the Public Summary if this field contains a string and is not empty i.e "" otherwise, it will return Synposis instead.

The same principle applies for PublicSummaryTitle and Title 

I then store the correct Title or Summary into new columns called Title and Description

In [41]:
def public_sum_conditional_remove(row):
    if isinstance(row['Public Summary'], str) and row['Public Summary'] != "":
        return row['Public Summary']
    else:
        return row['Family Synopsis']

In [42]:
GrantFamily_AppsGrant_Merged_RF_Filtered['Description'] = GrantFamily_AppsGrant_Merged_RF_Filtered.apply(public_sum_conditional_remove, axis=1)



/var/folders/ym/nt50cg2d4z79v6hgwgns0cvr0000gq/T/ipykernel_11455/286384530.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GrantFamily_AppsGrant_Merged_RF_Filtered['Description'] = GrantFamily_AppsGrant_Merged_RF_Filtered.apply(public_sum_conditional_remove, axis=1)


In [43]:
def public_title_conditional_remove(row):
    if isinstance(row['Public Summary Title'], str) and row['Public Summary Title'] != "":
        return row['Public Summary Title']
    else:
        return row['Family Title']

In [44]:
GrantFamily_AppsGrant_Merged_RF_Filtered['Title'] = GrantFamily_AppsGrant_Merged_RF_Filtered.apply(public_title_conditional_remove, axis=1)

/var/folders/ym/nt50cg2d4z79v6hgwgns0cvr0000gq/T/ipykernel_11455/1862147656.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GrantFamily_AppsGrant_Merged_RF_Filtered['Title'] = GrantFamily_AppsGrant_Merged_RF_Filtered.apply(public_title_conditional_remove, axis=1)


#### Creating additional columns
- e.g. Selecting Public title and/or summary , months 

##### Months
Adding the number of months a grant has been active for, may prove useful for others to get general idea of how long some grants can last.

In [45]:
GrantFamily_AppsGrant_Merged_RF_Filtered['Duration:Months'] =  GrantFamily_AppsGrant_Merged_RF_Filtered['Planned Dates:End Date'].dt.to_period('M').astype(int) - GrantFamily_AppsGrant_Merged_RF_Filtered['Planned Dates:Start Date'].dt.to_period('M').astype(int)

 
GrantFamily_AppsGrant_Merged_RF_Filtered['Duration:Months']

/var/folders/ym/nt50cg2d4z79v6hgwgns0cvr0000gq/T/ipykernel_11455/4289110183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GrantFamily_AppsGrant_Merged_RF_Filtered['Duration:Months'] =  GrantFamily_AppsGrant_Merged_RF_Filtered['Planned Dates:End Date'].dt.to_period('M').astype(int) - GrantFamily_AppsGrant_Merged_RF_Filtered['Planned Dates:Start Date'].dt.to_period('M').astype(int)


4         57
6         11
7          3
13        12
16        12
          ..
112007    59
112009     7
112013    23
112044    35
112452    96
Name: Duration:Months, Length: 16808, dtype: int64

In [46]:
final_df = GrantFamily_AppsGrant_Merged_RF_Filtered[['Financial Year', 'Internal ID',  'Lead Applicant', 'Recipient Org:Name', 'Recipient Org:Country', 'Region',  'Other Applicant(s)', 'Grant Programme:Title',  'Title', 'Description','Funding Org:Department', 'Funding Org:Team', 'Planned Dates:Start Date', 'Planned Dates:End Date', 'Currency', 'Amount Awarded', 'Partnership Name', 'For Regrant Type', 'Partnership Value', 'Award Date', 'Identifier', 'Recipient Org:Identifier', 'Funding Org:Name','Funding Org:Identifier']]

### 360Giving data request complete! 🎉

Well done! You've completed the FCAA data request - the final cell below transforms the dataframe into a csv file called 360Givingdraft - feel free to change the file name by editing "360Givingdraft.csv" in the cell below 

In [47]:
final_df.to_csv("360Givingdraft.csv",index=False )

In [48]:
final_df

,Financial Year,Internal ID,Lead Applicant,Recipient Org:Name,Recipient Org:Country,Region,Other Applicant(s),Grant Programme:Title,Title,Description,...,Currency,Amount Awarded,Partnership Name,For Regrant Type,Partnership Value,Award Date,Identifier,Recipient Org:Identifier,Funding Org:Name,Funding Org:Identifier
4,2015/16,203968/Z/16/Z,Mr Istvan Kleijn,Imperial College London,United Kingdom,Greater London,,PhD Studentship (Basic),Imperial College London - Theoretical Systems ...,Imperial College London - Theoretical Systems ...,...,GBP,141554.46,Not Applicable,None,141554.46,2016-09-30,360G-Wellcome-203968_Z_16_Z,360G-Wellcome-ORG:United-Kingdom,The Wellcome Trust,GB-CHC-210183
6,2015/16,205267/Z/16/Z,Prof Paul Workman,Institute of Cancer Research,United Kingdom,Greater London,,Open Access Award,Open access block grant 2016/17,None,...,GBP,63600.00,The Charity Open Access Fund,None,291107.16,2016-09-30,360G-Wellcome-205267_Z_16_Z,360G-Wellcome-ORG:United-Kingdom,The Wellcome Trust,GB-CHC-210183
7,2016/17,208626/Z/17/Z,Prof Trevor McMillan,Keele University,United Kingdom,West Midlands,,Vacation Scholarships,Vacation Scholarships 2017 - Keele University,Vacation Scholarships 2017 - Keele University,...,GBP,1500.00,Not applicable,None,1500.00,2017-06-16,360G-Wellcome-208626_Z_17_Z,360G-Wellcome-ORG:United-Kingdom,The Wellcome Trust,GB-CHC-210183
13,2017/18,209755/Z/17/Z,Prof Leszek Borysiewicz,University of Cambridge,United Kingdom,East of England,,Open Access Award,Open Access Awards 2017/18,None,...,GBP,650034.96,The Charity Open Access Fund,None,924482.96,2018-09-30,360G-Wellcome-209755_Z_17_Z,360G-Wellcome-ORG:United-Kingdom,The Wellcome Trust,GB-CHC-210183
16,2018/19,215036/Z/18/Z,Prof Alice Gast,Imperial College London,United Kingdom,Greater London,,Open Access Award,Open Access (COAF) Award 2018/19,None,...,GBP,445323.37,The Charity Open Access Fund,None,577449.37,2019-09-30,360G-Wellcome-215036_Z_18_Z,360G-Wellcome-ORG:United-Kingdom,The Wellcome Trust,GB-CHC-210183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112007,2021/22,226727/Z/22/Z,Prof Neil Ferguson,Imperial College London,United Kingdom,Greater London,"Dr Katy Gaythorpe, Dr Ilaria Dorigatti, Dr Car...",Discretionary Award,Vaccine Impact Modelling Consortium (VIMC 2.0)...,The Vaccine Impact Modelling Consortium (VIMC)...,...,GBP,3120676.00,None,None,3120676.00,2022-09-30,360G-Wellcome-226727_Z_22_Z,360G-Wellcome-ORG:United-Kingdom,The Wellcome Trust,GB-CHC-210183
112009,2021/22,226734/Z/22/Z,Mr David Glauser,Panorama Global,United States,International,,Discretionary Award,Civil Society Consultation and Engagement in t...,Lead CSO consultation and engagement in the de...,...,GBP,215795.81,None,None,215795.81,2022-09-30,360G-Wellcome-226734_Z_22_Z,360G-Wellcome-ORG:United-States,The Wellcome Trust,GB-CHC-210183
112013,2021/22,226738/Z/22/Z,Ms Desta Lakew,Amref Health Africa,Kenya,International,,Discretionary Award,"Thought Leadership, Advocacy, and Partnership ...","Increasing temperatures and sea levels, changi...",...,GBP,302843.76,None,None,302843.76,2022-09-30,360G-Wellcome-226738_Z_22_Z,360G-Wellcome-ORG:Kenya,The Wellcome Trust,GB-CHC-210183
112044,2021/22,226767/Z/22/Z,Miss Ekaterina Roth,World Economic Forum,Switzerland,International,,Discretionary Award,World Economic Forum Strategic Partnership Grant,None,...,GBP,1981331.45,None,None,1981331.45,2022-09-30,360G-Wellcome-226767_Z_22_Z,360G-Wellcome-ORG:Switzerland,The Wellcome Trust,GB-CHC-210183
